In [1]:
import numpy as np;
import pandas as pd;

In [2]:
trd = pd.read_csv("train_data.csv")
ted = pd.read_csv("test_data.csv")

In [3]:
len(trd)

10980

In [4]:
trd.head(2)

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)


In [5]:
train_data = np.array([(trd.loc[i]["text"],trd.loc[i]["airline_sentiment"]) for i in range(len(trd))])
test_data = np.array([ted.loc[i]["text"] for i in range(len(ted))])

In [6]:
test_data[0]

"@AmericanAir In car gng to DFW. Pulled over 1hr ago - very icy roads. On-hold with AA since 1hr. Can't reach arpt for AA2450. Wat 2 do?"

In [7]:
from nltk.stem import WordNetLemmatizer
lmt = WordNetLemmatizer()
from nltk.corpus import wordnet
def get_simple_pos(tag): #creating simple tags to pass into the lemmatizer
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [8]:
from nltk.corpus import stopwords
import string
stops = stopwords.words('english') + list(string.punctuation)

In [9]:
from nltk.tokenize import word_tokenize

In [10]:
from nltk import pos_tag

In [11]:
def clean_data(text):
    words = word_tokenize(text)
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])                                 
            clean_word = lmt.lemmatize(w, get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [12]:
documents = [(clean_data(i),category) for i,category in train_data]

In [13]:
testing_data = [clean_data(i) for i in test_data]

In [14]:
all_words = []
for doc in documents:
    all_words += doc[0]

In [15]:
import nltk

In [46]:
pp = nltk.FreqDist(all_words)
cc = pp.most_common(3)
cc

[('flight', 3349), ('united', 3180), ('usairways', 2253)]

In [16]:
freq = nltk.FreqDist(all_words)                 #will retrurn a freq distribution object
common = freq.most_common(8000)
features = [i[0] for i in common] #choosing the top 8000 frequency words

In [17]:
def get_feature_dict(words): #will return true/false if the word in present in the document or not
    current_features = {}
    words_set = set(words)
    for w in features:
        current_features[w] = w in words_set
    return current_features

In [18]:
categories = [category for document, category in documents]

In [49]:
t_d = [(get_feature_dict(doc),category) for doc, category in documents] # here data os in nltk classifier input data format

In [19]:
training_data = [get_feature_dict(doc) for doc, category in documents]

In [20]:
testing_data = [get_feature_dict(doc) for doc in testing_data] 

In [51]:
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(t_d)

In [53]:
classifier.show_most_informative_features(15) # most informative feature words

Most Informative Features
                   kudos = True           positi : negati =     51.9 : 1.0
               fantastic = True           positi : negati =     34.2 : 1.0
                favorite = True           positi : negati =     34.2 : 1.0
             outstanding = True           positi : negati =     29.1 : 1.0
                 helpful = True           positi : neutra =     26.3 : 1.0
                   flyfi = True           positi : negati =     24.1 : 1.0
                   thank = True           positi : negati =     22.6 : 1.0
                 awesome = True           positi : negati =     22.6 : 1.0
                    hold = True           negati : positi =     22.3 : 1.0
               beautiful = True           positi : negati =     21.5 : 1.0
                      hr = True           negati : positi =     21.4 : 1.0
                   daily = True           neutra : negati =     20.6 : 1.0
                  street = True           neutra : negati =     20.6 : 1.0

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
categories = [category for document, category in documents]

In [23]:
text_documents = [" ".join(document) for document, category in documents]

In [24]:
train_documents = [" ".join(document) for document in testing_data]

In [25]:
count_vec = CountVectorizer(max_features = 5000,max_df=0.8,ngram_range=(1,2))
x_train_features = count_vec.fit_transform(text_documents)

In [26]:
x_test_features = count_vec.transform(train_documents)

In [27]:
x_train = x_train_features.todense()
y_train = categories
x_test = x_test_features.todense()

In [28]:
features = count_vec.get_feature_names()

In [29]:
len(features)

5000

In [34]:
from sklearn.naive_bayes import MultinomialNB

In [35]:
clf = MultinomialNB()
clf.fit(x_train,y_train)
pred3 = clf.predict(x_test)
with open('predictions_twitter3.csv', 'w') as f:
    for item in pred3:
        f.write("%s\n" % item)